In [1]:
import json
import requests as re
import pandas as pd

In [2]:
def dados(tipo,id):
    response = re.get("https://pokeapi.co/api/v2/"+ tipo +"/" + (id))
    return response.json()

    
def dados_pokemon_all():
    response = re.get("https://pokeapi.co/api/v2/pokemon/")
    print(response.status_code)
    return response.json()

def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def select_colunas(df):
    df = pd.DataFrame(df)
    df = df.columns * (df.applymap(type) == str).all(0)
    df.replace(regex="",value=float("NaN"), inplace=True)
    df.dropna(axis="index", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.tolist()
    return df
    
def select_valores(df):
    df = pd.DataFrame(df)
    df = df.columns * (df.applymap(type) == int).all(0)
    df.replace(regex="",value=float("NaN"), inplace=True)
    df.dropna(axis="index", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.tolist()
    return df

In [3]:
dimensao_pokemon= pd.DataFrame([])
for id in range(1,21):
    linha_pokemon = dados("pokemon",str(id))
    pokemon_apoio_dimensao = pd.DataFrame(flatten_data(linha_pokemon),index=flatten_data(linha_pokemon).keys())
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "sprite")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "moves")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "game")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "is_hidden")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "is_default")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "slot")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "effort")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "forms")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(list(pokemon_apoio_dimensao.filter(regex = "held")), axis = 1, inplace = True)
    pokemon_apoio_dimensao.drop(["id","order","name","species_name","species_url",], axis = 1, inplace = True)
    pokemon_apoio_dimensao["ID_FATO"]="https://pokeapi.co/api/v2/pokemon/"+ str(id) + "/"
    dimensao_pokemon= pd.concat([dimensao_pokemon,pokemon_apoio_dimensao])


In [4]:
col_pokemon =list(dimensao_pokemon.columns)

for x in range (0,6):
    col_pokemon[col_pokemon.index("stats_"+ str(x) +"_base_stat")] = dimensao_pokemon["stats_"+ str(x) +"_stat_name"][0]

dimensao_pokemon.columns = col_pokemon
dimensao_pokemon.drop(list(dimensao_pokemon.filter(regex = "stats")), axis = 1, inplace = True)


In [5]:
Valores_dict = {}
for x in select_valores(dimensao_pokemon):
     Valores_dict[x] = "mean"

dimensao_pokemon = dimensao_pokemon.groupby(select_colunas(dimensao_pokemon)).agg(Valores_dict).reset_index()


In [7]:
fato = pd.DataFrame(dados_pokemon_all()["results"])
fato.columns = ["pokemon","ID_FATO"]
analitica= fato.merge(dimensao_pokemon, how='left', on="ID_FATO")
analitica.set_index("ID_FATO", inplace = True)


200
